In [2]:
# Import all packages and library

# Import package to scan hyperparameter
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

# Import package to reprocess the data
import numpy as np
import pandas as pd
import random

# Import properties from keras
from keras.models import *
from keras.layers import *
from keras import regularizers
from keras import initializers

# Import keras items
from keras.optimizers import Adam, Adadelta, SGD
from keras.activations import relu, sigmoid
from keras.layers.advanced_activations import PReLU
from keras.losses import binary_crossentropy
from keras.layers.normalization import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [3]:
# Get all of the data and reprocess them

# Get the reprocessed data from .npy file
x_train = np.load('../r-scripts/getting-data-current/data-sets/x_train.npy')
y_train = np.load('../r-scripts/getting-data-current/data-sets/y_train.npy')

x_dev = np.load('../r-scripts/getting-data-current/data-sets/x_val.npy')
y_dev = np.load('../r-scripts/getting-data-current/data-sets/y_val.npy')

x_test = np.load('../r-scripts/getting-data-current/data-sets/x_test.npy')
y_test = np.load('../r-scripts/getting-data-current/data-sets/y_test.npy')

# This Section is used to shuffle the data

# This Section is used to shuffle the data

# Aggregates elements
data_training = list(zip(x_train, y_train))
data_development = list(zip(x_dev, y_dev))
data_testing = list(zip(x_test, y_test))

# Shuffle the aggragated element on the list
random.shuffle(data_training)
random.shuffle(data_development)
random.shuffle(data_testing)

# Combine data training dan data development become one list of data train

data_train = data_training + data_development

# Split the shuffled data
x_train, y_train = zip(*data_train)
x_test, y_test = zip(*data_testing)

# Unpack the tuples
x_train = np.array(list(x_train))
y_train = np.array(list(y_train))
x_test = np.array(list(x_test))
y_test = np.array(list(y_test))

# Reshape the datasets
# x_train = x_train.reshape(615, 4034 * 20)
# x_test = x_test.reshape(150, 4034 * 20)

In [4]:
x_train.shape

(615, 4034, 20)

In [ ]:
model = Sequential()
model.add(Conv1D(32, 3, activation = "relu", input_dim =  20))
model.add(MaxPooling1D(3))
model.add(Conv1D(32, 3, activation = "relu"))
model.add(GRU(32, dropout = 0.1, recurrent_dropout = 0.5))
# model.add(layers.GRU(32, dropout = 0.1, recurrent_dropout = 0.5))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

model.compile(optimizer = 'sgd',
                    loss = 'binary_crossentropy',
                    metrics = ['accuracy'])
history = model.fit(x = x_train, 
                          y = y_train, 
                          epochs = 20, 
                          batch_size = 32, 
                          validation_split = 0.2, 
                          verbose = 1)

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, activation="relu", input_shape=(None, 20))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_13 (Conv1D)           (None, None, 32)          1952      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, None, 32)          3104      
_________________________________________________________________
gru_7 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dense_13 (Dense)             (None, 128)               4224      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 129       
Total params: 15,649
Trainable params: 15,649
Non-trainable params: 0
_________________________________________________________________
Train 

In [ ]:
# Pass the model design to KerasClassifier() wrapper -------------------------------

model = KerasClassifier(build_fn = build_fc_model,
	                         verbose = 1)

# Define the parameters that will be tuned randomly
keras_param_options = {
                       # Hyperparameters as parts of model designs (building blocks)
                       'input_num_hidden_units': [4, 8, 16],
                       'num_hidden_layers': [[0], [1], [2]],
                       'activation_function': ['relu'],
                       # Hyperparameters as part of optimization and regularization of the models
                       'optim_methods' : ['Adadelta', 'SGD'],
                       'l2_rate':[0.001],
                       'input_dropout_rates': [0.5],
                       'dropout_rates': [0.5],
                       'batch_norm' : ['no'],
                       # Fitting parameters
                       'batch_size': [8, 16, 32],
                       'epochs': [30],
                       'shuffle': [True]
                      }

# Using RandomizedSearchCV to find the best model randomly
random_search = GridSearchCV(model,
                            param_grid = keras_param_options,
                            n_jobs = 1,
                            cv = 5,
                            verbose = 10)

# Fit to the training data
random_search.fit(x_train, y_train)
df_result_hyper_tuned = pd.DataFrame.from_dict(random_search.cv_results_)
df_result_hyper_tuned.to_csv('result_hyper_tuned_model4_gridsearch.csv')
param_best_model_dict = dict(df_result_hyper_tuned['params'])
params = list(param_best_model_dict.values())

# Define function to fit the model
def train_fc_model(batch_sizes = None, num_epochs = None):
      model.fit(x = x_train,
                y = y_train,
                batch_size = batch_sizes,
                epochs = num_epochs,
                verbose = 1,
                shuffle = 1)

# Define the function to calculate sensitivity and specificity
def sensitivity_specificity(predictions, y_test, mode='binary'):
    if mode == 'binary':
        # Determine positive class predictions
        index = predictions > 0.5
        predictions = np.zeros(predictions.shape)
        predictions[index] = 1
        # No need to modify y_test since it consists of zeros and ones already
    else:
        y_test = y_test
        predictions = np.argmax(predictions, axis=-1)

    # In the binary classification case as we create, we can extract tn, fp, fn, tp as follows
    tn, fp, fn, tp = confusion_matrix(y_test, predictions, labels = [0, 1]).ravel()

    # Sensitivity = TP / (TP + FN)
    sensitivity = tp / (tp + fn)

    # Specificity = TN / (TN + FP)
    specificity = tn / (tn + fp)

    # Precision = TP / (TP + FP)
    precision = tp / (tp + fp)

    # Return sensitivity, specificity, precision
    return(sensitivity, specificity, precision)


# Define function to evaluate and predict
def evaluate_predict_fc_model():
  loss, acc = model.evaluate(x_test, y_test, verbose = 0)
  prediction = model.predict(x_test)
  sensitivity, specificity, precision = sensitivity_specificity(prediction, y_test, mode='binary')
  return acc, sensitivity, specificity, precision

# make prediction
result_list = []
columns_names = ['Parameters',
                'Accuracy',
                'Sensitivity',
                'Specifity']


for i in range(len(params)):
    list_par = list(params[i].values())
    model = build_fc_model(input_num_hidden_units = list_par[6],
                           num_hidden_layers = list_par[8],
                           activation_function = list_par[0],
                           l2_rate = list_par[7],
                           input_dropout_rates = list_par[5],
                           dropout_rates = list_par[3],
                           optim_methods = list_par[9],
                           batch_norm = list_par[1])
    train_fc_model(batch_sizes = list_par[2], num_epochs = list_par[4])
    acc, sensitivity, specifity, precision = evaluate_predict_fc_model()
    result_line = np.array((params[i],
                            acc,
                            sensitivity,
                            specifity))
    result_list.append(result_line[:])
    result_array = np.asarray(result_list)

    df_results = pd.DataFrame(result_array,
                         columns = columns_names)

df_results.to_csv('df_result_prediction_model4_gridsearch.csv')

Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] activation_function=relu, batch_norm=no, batch_size=8, dropout_rates=0.5, epochs=30, input_dropout_rates=0.5, input_num_hidden_units=4, l2_rate=0.001, num_hidden_layers=[0], optim_methods=Adadelta, shuffle=True 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4)                 322724    
_________________________________________________________________
dropout_1 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 322,729
Trainable params: 322,729
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
492/492 [==============================] - 3s 6ms/step - loss: 0.6979 - acc: 0.5163
Epoch 2/30
492/492 [==============================] - 2s 3ms/step - loss: 0.6092 - acc: 0.6037
Epoch 3/30
492/492 [==============================] - 2s 3ms/step - loss: 0.5276 - acc: 0.6829
Epoch 4/30
492/492 [==============================] - 2s 3ms/step - loss: 0.4307 - acc: 0.7683
Epoch 5/30
492/

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   51.9s remaining:    0.0s


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 4)                 322724    
_________________________________________________________________
dropout_2 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total params: 322,729
Trainable params: 322,729
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
492/492 [==============================] - 2s 5ms/step - loss: 0.6956 - acc: 0.5630
Epoch 2/30
492/492 [==============================] - 1s 3ms/step - loss: 0.6005 - acc: 0.7053
Epoch 3/30
492/492 [==============================] - 1s 3ms/step - loss: 0.5022 - acc: 0.7419
Epoch 4/30
492/492 [==============================] - 1s 3ms/step - loss: 0.4525 - acc: 0.7520
Epoch 5/30
492/

...

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 4)                 322724    
_________________________________________________________________
dropout_8 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 5         
Total params: 322,729
Trainable params: 322,729
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
492/492 [==============================] - 2s 4ms/step - loss: 0.7012 - acc: 0.5264
Epoch 2/30
492/492 [==============================] - 1s 2ms/step - loss: 0.6582 - acc: 0.5711
Epoch 3/30
492/492 [==============================] - 1s 2ms/step - loss: 0.5954 - acc: 0.6362
Epoch 4/30
492/492 [==============================] - 1s 2ms/step - loss: 0.5326 - acc: 0.7236
Epoch 5/30
492/

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  5.5min remaining:    0.0s


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 4)                 322724    
_________________________________________________________________
dropout_9 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 5         
Total params: 322,729
Trainable params: 322,729
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
492/492 [==============================] - 2s 4ms/step - loss: 0.7009 - acc: 0.5305
Epoch 2/30
492/492 [==============================] - 1s 2ms/step - loss: 0.6822 - acc: 0.5874
Epoch 3/30
492/492 [==============================] - 1s 2ms/step - loss: 0.6556 - acc: 0.6199
Epoch 4/30
492/492 [==============================] - 1s 2ms/step - loss: 0.6149 - acc: 0.6565
Epoch 5/30
492/

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  6.0min remaining:    0.0s


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 4)                 322724    
_________________________________________________________________
dropout_10 (Dropout)         (None, 4)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 5         
Total params: 322,729
Trainable params: 322,729
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30
492/492 [==============================] - 2s 4ms/step - loss: 0.7018 - acc: 0.5000
Epoch 2/30
492/492 [==============================] - 1s 2ms/step - loss: 0.6682 - acc: 0.6240
Epoch 3/30
492/492 [==============================] - 1s 2ms/step - loss: 0.6067 - acc: 0.7053
Epoch 4/30
492/492 [==============================] - 1s 2ms/step - loss: 0.5442 - acc: 0.7541
Epoch 5/30
492/